# SRD 1 praca domowa

<b> Termin oddania: 04.04.2019</b>

Rozwiązane zadania można wysyłać w 2 formatach. Albo jako skrypt <tt>.jl</tt>  z komentarzami określającymi numer rozwiązywanego zadania, albo jako uzupełniony notatnik z poleceniami (format <tt>.ipynb</tt>) razem z wygenerowanym na jego podstawie plikiem <tt>.html</tt>. Można to zrobić drogą mailową wysyłając plik lub pliki na adres [bpankra@sgh.waw..pl](mailto: bpankra@sgh.waw.pl)

## Zadanie 1

Napisz funkcję, która dla dowolnego wektora długości $n$ wygeneruje [Macierz Vandermonde’a](https://pl.wikipedia.org/wiki/Macierz_Vandermonde%E2%80%99a) <b>(3 p.)</b>

In [1]:
function Vandermonde_Matrix(vector::Array{Int64,1})
    matrix = ones(length(vector),length(vector)) # generacja macierzy o długościach takich samych jak wejściowy wektor
    for i in 1:length(vector)
        matrix[i,2] = vector[i,1] # input wejściowego wektora do drugiej kolumny macierzy
        for j in 3:length(vector)
            matrix[i,j] = (matrix[i,2])^(j-1) # kalkulacja kolejnych potęg
        end
    end
    return matrix
end
Vandermonde_Matrix([0,2,4,8,1,2]) # wywołanie funkcji

6×6 Array{Float64,2}:
 1.0  0.0   0.0    0.0     0.0      0.0
 1.0  2.0   4.0    8.0    16.0     32.0
 1.0  4.0  16.0   64.0   256.0   1024.0
 1.0  8.0  64.0  512.0  4096.0  32768.0
 1.0  1.0   1.0    1.0     1.0      1.0
 1.0  2.0   4.0    8.0    16.0     32.0

## Zadanie 2

Korzystając z odpowiednich bibliotek i wbudowanych funkcji napisz funkcję, która generuje macierz trójdiagonalną o rozmiarze $n \times n$ i [postaci](https://en.wikipedia.org/wiki/Toeplitz_matrix):

\begin{bmatrix}
4 & -1 & 0 & 0 & \dots & 0 \\
-1 &  4 & -1 & 0 & \dots  & 0 \\
0 &  -1 & 4  & -1 & \dots & 0 \\
\vdots & \vdots & \vdots  & \vdots & \ddots & \vdots \\
0 & 0 & 0 & 0 & \dots & 4 \\
\end{bmatrix}

Następnie napisz kod, który dla macierzy z $A$ wygenerowanej za pomocą funkcji z poprzedniego zadania o wymiarach $n \in [10,100,200,500,700,1000,2000,5000,7000,10000]$ wyznaczy rozwiązanie równania $Ax = b$, gdzie $b$ jest wektorem jedynek o długości $n$. Wykorzystaj do tego trzy metody:

- wbudowaną w pakiet [<tt>LinearAlgebra</tt>](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/index.html) funkcję [<tt>Cholesky</tt>](https://en.wikipedia.org/wiki/Cholesky_decomposition)
- operator dzielenia macierzy <tt>A\b</tt> 
- mnożenie wektora przez odwrotność macierzy: <tt>$A^{-1}b$</tt>

Przedstaw na wykresie czas działania tych trzech metod w zależności od $n$. <b>(4 p.)</b>

In [2]:
# import i instalacja potrzebnych bibliotek
import Pkg
Pkg.add("TimesDates")
Pkg.add("LinearAlgebra")
Pkg.add("Plots")
#using BenchmarkTools
using TimesDates, Dates
using LinearAlgebra
using Plots
plotly()

 Resolving package versions...
  Updating `~/.julia/Project.toml`
 [no changes]
  Updating `~/.julia/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/Project.toml`
 [no changes]
  Updating `~/.julia/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/Project.toml`
 [no changes]
  Updating `~/.julia/Manifest.toml`
 [no changes]


Plots.PlotlyBackend()

In [3]:
# po diagonali liczba 4, obok diagonali -1
function Tridiagonal_Matrix(n)
    matrix = zeros(n,n)
    for i in 1:n
        matrix[i,i] = 4
    end
    for i in 1:(n-1)
        matrix[i+1,i] = -1
        matrix[i,i+1] = -1
    end
    return matrix
end
Tridiagonal_Matrix(10)

10×10 Array{Float64,2}:
  4.0  -1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   4.0  -1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   4.0  -1.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   4.0  -1.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   4.0  -1.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   4.0  -1.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0   4.0  -1.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0  -1.0   4.0  -1.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0  -1.0   4.0  -1.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  -1.0   4.0

In [4]:
# wykonanie macierzy trzema metodami
# czas wykonania jest zliczany do array y
y = zeros(10,3)
i = 1
for n in [10,100,200,500,700,1000,2000,5000,7000,10000]
    A = Tridiagonal_Matrix(n)
    b = ones(n,1)
    start = now(); cholesky(A)\b; stop = now()
    y[i,1] = Dates.value(Dates.Millisecond(stop - start))
    start = now(); A\b; stop = now()
    y[i,2] = Dates.value(Dates.Millisecond(stop - start))
    start = now(); A^(-1)*b; stop = now()
    y[i,3] = Dates.value(Dates.Millisecond(stop - start))
    i += 1
end

In [5]:
# wykres
x = ["10","100","200","500","700","1000","2000","5000","7000","10000"]
plot(x,y,title="Czas obliczeń [ms]",label=["Cholesky" "A\\b" "A^(-1)*b"],lw=3)

## Zadanie 3

Napisz funkcję, która obliczy wartość liczby $\pi$ metodą Monte Carlo <b>(3 p.)</b>

In [6]:
# losujemy punkty na planie kwadratu o boku 1
# i sprawdzamy czy punkt trafia w koło o środku w punkcie 0.5;0.5 i promieniu 0.5
# (koło jest styczne do wszystkich boków kwadratu)
# powód: pole kwadratu to (2r)² gdzie r to promień koła wewnątrz kwadratu, a pole koła to πr²
# proporcje tych pól to πr²/(2r)²  ->  πr²/4r²  ->  πr²/4r²  ->  π/4 
# z tego wynika, że szansa trafienia w koło przybliża pi podzielone przez 4

function PI_Monte_Carlo(iterations::Int64)
    # licznik trafień
    counter = 0
    # losujemy tyle razy ile wynosi wartość argumentu iterations
    for i = 1:iterations
        # losuje punkt wewnątrz kwadratu i liczy jego odległość do środka koła
        distance = ((0.5-rand())^2+(0.5-rand())^2)^(1/2)
        # jeśli punkt jest w odległości mniejszej bądź równej niż 0.5 to znajduje się na kole
        if distance <= 0.5
            counter += 1 # inkrementacja licznika trafień
        end
    end
    # losowanie przybliża π podzielone przez 4 więc tutaj mnożymy przez 4 żeby otrzymać π
    return (counter/iterations)*4
end
PI_Monte_Carlo(1000000000) # wywołanie funkcji z miliardem iteracji

3.141556476